In [1]:
import cv2
import numpy as np


#Initiating the camera
cap= cv2.VideoCapture(0)

#Infinite loop for capturing the frames
while(True):
	ret,img = cap.read()

	#Reshaping teh image to be fed into the kmeans data
	Z = img.reshape((-1,3))
	#Converting it to float data type
	Z = np.float32(Z)

	# define criteria, number of clusters(K) and apply kmeans()
	criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 3, 0.9)
	#Number of clusters to be defined
	K = 12
	ret,label,center=cv2.kmeans(Z,K,None,criteria,2,cv2.KMEANS_RANDOM_CENTERS)

	# Now convert back into uint8, and make original image
	center = np.uint8(center)
	res = center[label.flatten()]
	res2 = res.reshape((img.shape))
	#increasing the intensity for brighter image
	res2 = res2+15

	cv2.imshow("Cartoonised", res2)
	#Exiting the application on press of q
	if(cv2.waitKey(9)==ord('q')):
		break

cap.release()
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy


#Initiating the camera
cap= cv2.VideoCapture(0)

#Infinite loop for capturing the frames
while(True):
	ret,frame = cap.read()
	
	#converting the image to greyscale for edge detection
	image_bw = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
	# Applying the bilateral filter for smoothening the image without edge loss  
	image_bw = cv2.bilateralFilter(image_bw, -1,10, 1)
	image_bw = cv2.GaussianBlur(image_bw, (5,5), 0)
	
	#Getting the edges of the image using Laplacian
	edges = cv2.Laplacian(image_bw,-1,ksize=5)
	#Threshholding the edges for the mask to be blended
	_,mask = cv2.threshold(edges,30, 255, cv2.THRESH_BINARY_INV)

	#taking the impurities out of the mask using median blur
	for i in range(13):
		#Here the kernel size can be varied according to the resolution of camera 
		mask= cv2.medianBlur(mask,3)

	#Eroding and dilating the mask for making it look realistic 
	kernel = numpy.ones((3,3),numpy.uint8)
	mask = cv2.erode(mask, kernel)
	mask =cv2.erode(mask, kernel)
	mask = cv2.dilate(mask, kernel)
	
	#Final task of splitting the frame into three layers(b,g,r) and application of mask
	image_b, image_g, image_r = cv2.split(frame)

	final_b = cv2.bitwise_and(image_b, mask)
	final_g = cv2.bitwise_and(image_g, mask)
	final_r = cv2.bitwise_and(image_r, mask)

	final = cv2.merge((final_b,final_g,final_r))

	cv2.imshow("fine", final)
	#Exiting the application on press of q
	if(cv2.waitKey(2)==ord('q')):
		break

cap.release()
cv2.destroyAllWindows()